In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/repos/Epilepsy_Microglia
%pip install -q -r requirements.txt

Mounted at /content/drive
/content/drive/MyDrive/repos/Epilepsy_Microglia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.5/517.5 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━

**2. Preprocess**

0. Check var_names, make names unique
1. QC
2. Setup SCVI model


In [3]:
import torch

# Single Cell Libraries
import scvi
import scanpy as sc
import anndata as ad

# Data Processing and Plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import igraph
import leidenalg

# File grab
import os
import tempfile
import pooch
import shutil, subprocess, glob
import gzip

print(torch.__version__)
print(scvi.__version__)
print(torch.cuda.is_available())

2.8.0+cu126
1.4.0
True


In [4]:
# not for github

RAW_DIR = "/content/drive/MyDrive/datas/epilepsy_microglia/raw/GSE201048_raw/"
OUT_DIR = "/content/drive/MyDrive/datas/epilepsy_microglia/processed/GSE201048"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# not for github

tempdir = "/content/plots/"
os.makedirs(tempdir, exist_ok=True)

In [8]:
# not for github

adata = sc.read_h5ad(os.path.join(OUT_DIR, "kumar.h5ad"))
adata

AnnData object with n_obs × n_vars = 90005 × 25206
    obs: 'patient_id', 'sample_id', 'dataset', 'sex', 'age', 'dx', 'dx_subtype', 'region', 'hemisphere', 'procedure', 'protocol', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', '_scvi_batch', '_scvi_labels'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'log1p'
    obsm: '_scvi_extra_categorical_covs', '_scvi_

In [ ]:
# not for github
adata.write(os.path.join(OUT_DIR, "kumar.h5ad"))

In [9]:
# not for github
adata.var

,gene_ids,feature_types,mt,ribo,hb,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,highly_variable,highly_variable_rank,means,variances,variances_norm
RP11-34P13.7,ENSG00000238009,Gene Expression,False,False,False,301,0.003409,0.003404,99.666818,308.0,5.733341,301,False,NaN,0.003422,0.003566,0.820649
RP11-34P13.8,ENSG00000239945,Gene Expression,False,False,False,37,0.000432,0.000432,99.959044,39.0,3.688879,37,False,NaN,0.000433,0.000478,0.980844
AL627309.1,ENSG00000237683,Gene Expression,False,False,False,7600,0.094896,0.090659,91.587430,8573.0,9.056490,7600,False,NaN,0.095250,0.111577,0.840760
RP11-34P13.9,ENSG00000241599,Gene Expression,False,False,False,19,0.000210,0.000210,99.978969,19.0,2.995732,19,False,NaN,0.000211,0.000211,0.913050
AP006222.2,ENSG00000228463,Gene Expression,False,False,False,6042,0.073234,0.070676,93.312007,6616.0,8.797398,6041,False,NaN,0.073496,0.082272,0.808608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL354822.1,ENSG00000215615,Gene Expression,False,False,False,224,0.002568,0.002565,99.752051,232.0,5.451038,224,False,NaN,0.002578,0.002749,0.855289
KIR2DL2,ENSG00000215764,Gene Expression,False,False,False,76,0.000919,0.000918,99.915874,83.0,4.430817,76,False,NaN,0.000922,0.001099,1.013862
AL590523.1,ENSG00000268276,Gene Expression,False,False,False,33,0.000365,0.000365,99.963472,33.0,3.526361,33,False,NaN,0.000367,0.000367,0.895671
PNRC2-1,ENSG00000215700,Gene Expression,False,False,False,3454,0.041321,0.040490,96.176708,3733.0,8.225235,3454,False,NaN,0.041475,0.046467,0.810687


In [ ]:
# not for github
!tar -czf GSE201048_raw.tar.gz /content/drive/MyDrive/datas/epilepsy_microglia/raw/GSE201048_raw

In [ ]:
adatas = {}

for file in glob.glob(os.path.join(RAW_DIR, "*.h5ad")):
  sample_adata = sc.read_h5ad(file)
  sample_adata.var_names_make_unique()
  sample_id = sample_adata.obs["sample_id"].iloc[0]
  adatas[sample_id] = sample_adata

adata = ad.concat(
    adatas.values(),
    label="sample_id",
    keys=adatas.keys(),
    merge="same"
)
adata.obs_names_make_unique()
print(adata.obs["sample_id"].value_counts())
adata

In [ ]:
# 0. Check var names (ensemble vs symbol)
print(adata.var_names[:10])

In [ ]:
# 1-1. QC
adata.var["mt"] = adata.var_names.str.startswith("MT-")
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True)

sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter = 0.4,
    multi_panel=True,
    save = "qc.png"
)
sc.pl.scatter(
    adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt",
    save = "qc.png"
)


In [ ]:
# 1-2.
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    layer="counts",
    flavor="seurat_v3",
)

adata

In [ ]:
# 2-1.
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="sample_id",
    categorical_covariate_keys=[
        "patient_id", "sample_id", "dataset", "sex", "dx", "dx_subtype",
        "region", "hemisphere", "procedure", "protocol"
    ],
    continuous_covariate_keys=[
        "age"
    ]
)

In [ ]:
# 2-2.
model = scvi.model.SCVI(adata)
model.train()

In [ ]:
# 2-3.
model_dir = os.path.join(OUT_DIR, "model")
os.makedirs(model_dir, exist_ok=True)
model.save(model_dir, save_anndata=False, overwrite=True)